# Search Engine for Swahili Text

Install dependencies

In [1]:
!pip install -q requests pandas scikit-learn jupyter transformers tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [2]:
import pandas as pd
import requests
import io
import numpy as np

## Get the data

Download the data. We shall be using Swahili news text data from the [masakhane-ner][0] project.

[0]: https://github.com/masakhane-io/masakhane-ner/tree/main/text_by_language/swahili

In [3]:
docs_url = 'https://raw.githubusercontent.com/masakhane-io/masakhane-ner/main/text_by_language/swahili/voa_clean_final.txt'
docs_response = requests.get(docs_url)
documents_raw = docs_response.text

We convert the documents into a pandas Dataframe. Each line in the raw format is a sentence from the VOA swahili version news articles. We use the StringIO object to extract line by line.

In [4]:
pd.set_option('display.max_colwidth', 999) # display full sentences

in_memory_file = io.StringIO(documents_raw)
df = pd.DataFrame([l.strip() for l in in_memory_file], columns=['documents'])
df.tail()

,documents
7667,"Wakati huohuo upande wa utetezi uliomwakilisha Jaji umesema hauna tena sababu ya kuwaita mashahidi wake, lakini itaomba mahakama hiyo kutupia mbali kesi hiyo kwa sababu serikali ya Nigeria imeshindwa kuthibitisha madai yake."
7668,"Baadhi ya wananchi wa Nigeria wanadai kuwa hatua iliyochukuliwa na Rais Buhari, ni njama ya kumweka mwengine kutoka upande wa Kaskazini mwa Nigeria kuchukuwa cheo hicho cha Jaji Mkuu."
7669,"Madai yao pia yanalenga suala la njama ya kurudi tena madarakani wakidai kuwa Buhari alikuwa akijitayarisha kushindania urais awamu ya pili na iwapo angeshindwa kesi ikifikishwa Mahakama Kuu kabisa, atakuwa na mtu wake wa karibu atakaye mwonea huruma na kuhakikisha kwamba anapata ushindi."
7670,"Imetayarishwa na Mwandishi wetu, Collins Atohengbe, Nigeria"
7671,"Walinzi wa pwani ya Libya wamekamata wahamiaji 400 waliokuwa wakonjiani katika pwani ya Mediterranean ya nchi hiyo wakielekea Ulaya na kuwarejesha katika mji mkuu wa Tripoli masaa 24 yaliyopita, Shirika la uhamiaji la Umoja wa Mataifa UN limesema Jumapili."


In [5]:
df.shape

(7672, 1)

The dataset consists of 7672 sentences in total.

## Basic Text Search

### Keyword Filtering

Filter documents that contain the word 'waziri' (minister).

In [6]:
df[df.documents.str.contains("corona", case=False)]

,documents
5,"Nchini Afrika kusini watu 145 zaidi, wameambukizwa virusi vya Corona, na kujumulisha idadi ya watu 2,173 ambao wameambukizwa virusi vya Corona nchini humo."
6,Taarifa ya wizara ya afya hata hivyo haijasema idadi ya watu ambao wamekufa wala kupona kutokana na virusi vya Corona nchini humo.
7,"Nchini Sudan, maafisa wameongeza mikakati zaidi ya kuzuia virusi vya Corona kusambaa."
9,Jumla ya watu 19 wamethibitishwa kuambukizwa virusi vya Corona nchini Sudan kusini.
41,"“Upimaji wetu wa virusi vya corona ni wa juu zaidi (vipimo 25 milioni) na ambavyo vya utaalam wa juu, na inatufanya tuonekane kama tuna maambukizi mengi zaidi, hasa ukilinganisha na nchi nyingine,” alituma ujumbe wa tweet Jumapili jioni."
...,...
7569,Hatua zilizofuatia ni sampuli ilichukuliwa na kupelekwa maabara ya taifa ya afya ya jamii iliyopo Dar es Salaam kwa ajili ya uchunguzi na kubainika mgonjwa huyo ana virusi vya Corona.
7572,"Kwa upande wake Mwakilishi Mkazi wa Shirika la Afya duniani (WHO) hapa nchini, Dk Tigest Katsela Mengestu, amesema ugonjwa wa Corona umekuwa tishio katika nchi mbalimbali, huku akiipongeza serikali ya Tanzania kutokana na hatua inazozichukua katika kukabiliana na ugonjwa huo."
7584,Hisia mseto ziliendelea kutolewa na wadau mbalimbali Jumanne baada ya kutangazwa kuahirishwa kwa michezo ya Olimpiki 2020 kutokana na janga la coronavirus.
7593,"Katika taarifa ya pamoja siku ya Jumanne, waandalizi wa Tokyo 2020 na IOC walisema: ""Kutokana na kasi ya mlipuko wa wa coronavirus hali kuwa mbaya duniani kote."


### Vectorization

#### Count Vectorization

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X = cv.fit_transform(df.documents)
X.shape

(7672, 19056)

In [8]:
names = cv.get_feature_names_out()
len(names), names[-7:]

(19056,
 array(['zulu', 'zuma', 'zungumza', 'zuri', 'zusha', 'zweli', 'évariste'],
       dtype=object))

In [9]:
df_docs = pd.DataFrame(X.toarray(), columns=names)
df_docs.head()

,00,000,002,01,02,023,041,043,052,069,...,zubaidi,zubeir,zuio,zulu,zuma,zungumza,zuri,zusha,zweli,évariste
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
first_doc = df_docs.loc[0]
first_doc[first_doc > 0]

14            1
19            1
afya          1
covid         1
imeripoti     1
jumatatu      1
kuwa          1
maambukizi    1
takriban      1
tanzania      1
wamepata      1
watu          1
wizara        1
ya            3
zaidi         1
Name: 0, dtype: int64

In [11]:
df.loc[0]

documents    Wizara ya afya ya Tanzania imeripoti Jumatatu kuwa, watu takriban 14 zaidi wamepata maambukizi ya Covid-19.
Name: 0, dtype: object

The word 'ya' is an example of a stop word i.e. relatively insignificant words that can skew the results.

##### Query-Document similarity

Representing the query in the same vector space.

In [12]:
query = "nchi za afrika zinajadiliana na shirika la afya kutafuta njia za kukabiliana na janga la corona"
q = cv.transform([query])
df_query = pd.DataFrame(q.toarray(), columns=names)
encoded_query = df_query.loc[0]
encoded_query[encoded_query > 0]

afrika         1
afya           1
corona         1
janga          1
kukabiliana    1
kutafuta       1
la             2
na             2
nchi           1
njia           1
shirika        1
za             2
Name: 0, dtype: int64

The more the common words and counts between query and document, the more similar they are. We use dot-product to calculate the score between each document and query, then rank by score. For example, dot-product between query and first document:

In [13]:
(first_doc * encoded_query).sum()

np.int64(1)

Across all the docs:

In [14]:
query_vector = q.toarray().flatten()
score = X.dot(query_vector)

In [15]:
score.argmax(), score[2433]

(np.int64(2433), np.int64(30))

The highest match is document at index 2443, with a score of 18. We can take a look at the document:

In [16]:
df.documents[2433]

'”  Swali la msingi la Mueller  Swali la msingi la Mueller, mkurugenzi wa zamani wa FBI, ambalo anatafuta majibu : Ni iwapo Trump na wasaidizi wake walishirikiana na Warusi kuchafua kampeni ya mgombea wa chama cha Demokrat Hillary Clinton, mwaka 2016, kwa kutuma barua pepe zenye kudhalilisha zilizoibiwa kutoka Kamati ya Taifa ya chama cha Demokrat na mwenyekiti wa kampeni ya Clinton? Au iwapo Trump alikuwa amenufaika bila ya kukusudia na mbinu chafu za Russia? Na iwapo rais alijaribu kuharibu uchunguzi uliofuatia ili kujilinda yeye mwenyewe na washauri wa kisiasa na wasaidizi wake?  Huu ndio ujumbe wa Idara ya Sheria kwa bunge la Congress juu ya hitimisho la uchunguzi uliofanywa na Mueller.'

Get the top 5 scores:

In [17]:
top_idx = np.argsort(-score)[:10]

In [18]:
df.iloc[top_idx]

,documents
2433,"” Swali la msingi la Mueller Swali la msingi la Mueller, mkurugenzi wa zamani wa FBI, ambalo anatafuta majibu : Ni iwapo Trump na wasaidizi wake walishirikiana na Warusi kuchafua kampeni ya mgombea wa chama cha Demokrat Hillary Clinton, mwaka 2016, kwa kutuma barua pepe zenye kudhalilisha zilizoibiwa kutoka Kamati ya Taifa ya chama cha Demokrat na mwenyekiti wa kampeni ya Clinton? Au iwapo Trump alikuwa amenufaika bila ya kukusudia na mbinu chafu za Russia? Na iwapo rais alijaribu kuharibu uchunguzi uliofuatia ili kujilinda yeye mwenyewe na washauri wa kisiasa na wasaidizi wake? Huu ndio ujumbe wa Idara ya Sheria kwa bunge la Congress juu ya hitimisho la uchunguzi uliofanywa na Mueller."
6107,"Lissu ameongeza kwamba “Kwa kutumia njia hizo za amri za kisiasa, vyombo vya ulinzi na usalama vikiwemo jeshi la wananchi la Tanzania, jeshi la polisi, taasisi ya kuzuia na kupambana na rushwa, na idara ya usalama wa taifa pamoja na mamlaka ya kodi TRA, vinatumika kukamata kwa nguvu na kutaifisha mali za wafanyabiashara wetu wa ndani na makampuni ya wawekezaji kutoka nje."
968,"Akizungumza na shirika la habari la AFP Akram Taher mumini moja amesema “Sherehe za Eid hazifani manmo wakati huu wa hali ya janga la corona – watu wanahisia ya kua na hofu” Bara la Asia Waislam katika bara la Asia – kutoka Indonesia hadi Pakistan, Malaysia na Afghanistan – wamekusanyika katika masoko katika shamrashamra za manunuzi ya sikukuu, wakikiuka muongozo wa kudhibiti virusi vya corona na wakati mwengine polisi wakijaribu kutawanya mikusanyiko ya makundi makubwa."
530,"Changamoto za uokozi Msemaji wa shirika la kimataifa la msalaba mwekundu Caroline Haga ameliambia shirika la habari la AFP kwamba wafanyakazi wa uokozi wanakabiliwa na changamoto kubwa kuwafikia wanaohitaji msaada na wana wasiwasi na huzuni, kwani siku ya Jumanne walifanikiwa kuwaokoa watu 167 na kwamba muda unakimbia haraka sana na watu bado wamekwama na wako hatarini."
842,"“Wakati hatuamini kuwa katika hatua hii, hali hiyo inahitaji kupitishwa azimio, kuna dalili zote za kututahadharisha kuwa mgogoro wa kuminywa kwa haki za binadamu unafukuta,” imesema barua hiyo, ambayo imesainiwa na Mtandao wa kutetea haki za binadamu wa Bara la Afrika, Shirika la Amnesty International, Shirika la ARTICLE 19, Shirika la Asian Forum for Human Rights and Development, Kituo cha Centre for Civil Liberties – Ukraine, Human Rights Watch na Tume ya International Commission of Jurists na taasisi nyingine."
5142,"Rais Trump Rais wa Marekani Donald Trump, anasema Biden, “amerusha baruti katika moto, na anajukumu la kutoa maelezo kwa watu wa Marekani juu ya mkakati na mpango wa kuhakikisha vikosi vyetu na wafanyakazi wa ubalozi, watu wetu na maslahi yetu, yote hapa nchini na nchi za nje, na washirika wetu katika eneo lote la Mashariki ya Kati na maeneo mengine."
5682,"” Ogwell amesema taasisi yake, ambayo ni shirika la ushauri wa kifundi la Umoja wa Afrika, anashirikiana na AU kuzijengea uwezo wa utayari nchi mbalimbali katika maeneo makuu matatu, ikiwemo kuboresha utoaji tahadhari katika bandari za nchi hizo na mahospitali; kuongeza utaalamu wa kuweza kupima kirusi COVID-19, ambao tayari nchi 43 wanauwezo huo; na kujenga uwezo wa kuzuia maambukizi na kudhibiti hali hiyo ili wagonjwa wenye maambukizi waweze kuwekewa karantini na kufuatiliwa."
4934,"Morales alisema : ""Kaka na dada zangu nchini Bolivia na ulimwenguni kote, nawafahamisha niko hapa na Makamu wa Rais na Waziri wa Afya, na baada ya kuwasilikiliza rafiki zangu kutoka shirikisho la vuguvugu la kijamii na shirikisho la umoja wa kibiashara na pia kwa kusikiliza Kanisa Katoliki, natangaza kujiuzulu wadhifa wangu wa urais."
7355,Kiongozi huyo wa cheo cha juu katika nchi za falme za kiarabu aliwaambia waandishi wa habari alisikiliza mtazamo wa Jenerali Abdel Fattah Burhan kuhusu matatizo ya Sudan na yeye alimweleza mtazamo wa umoja wa falme za kiarabu kuhusiana na hali hii ya kisiasa nchini Sudan Katibu mkuu wa umoja wa nchi za falme za kiara

Using cosine similarity instead:

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

score = cosine_similarity(X, q).flatten()
df.iloc[np.argsort(-score)[:10]]

,documents
5437,"Pamoja na kuwa wanashirikiana katika mipaka yao na kuwepo kwao katika wigo la kiuchumi la pamoja, nchi za Afrika Mashariki zinakuwa na hisia kali baina yao zinazotokana na tofauti zao za kiuchumi na kisiasa."
3022,"“Inasikitisha kwamba nchi kadhaa duniani na mashirika mbalimbali, ikiwemo Marekani, China na shirika la afya duniani, yanakabiliwa na tishio la maambukizi ya Corona."
4808,"Amesema ""mwanzoni tulizungumza kujifunza kutoka uzoefu wa nchi nyingine katika kupambana na janga la corona, sasa tunazungumzwa na nchi nyingine kama kielelezo hasi cha vita dhidi ya janga la corona Afrika na duniani."
664,"Amesema ""mwanzoni tulizungumza kujifunza kutoka uzoefu wa nchi nyingine katika kupambana na janga la corona, sasa tunazungumzwa na nchi nyingine kama kielelezo hasi cha vita dhidi ya janga la corona Afrika na duniani."
6369,"Mkurugenzi Mkuu wa Shirika la Afya Duniani Tedros Adhanom Ghebreyesus, ameonya kufungwa kwa mipaka ya nchi na kusitisha shughuli zote ili kupambana na janga la COVID 19 kunaweza kusababisha kuongezeka kwa vifo kutokana na ugonjwa wa Malaria katika nchi za Afrika."
4571,Shahidi mmoja aliliambia shirika la habari la Reuters katika wiki kadhaa za karibuni za maandamano yanayoipinga serikali yaliyochochewa na malalamiko ya kiuchumi na kisiasa.
6182,"Ndege za kijeshi za India, zilivuka mpaka na kuingia katika nchi jirani ya Pakisan na kutekeleza mashambulizi dhidi ya kambi iliyodaiwa kuwa ya kutoa mafunzi kwa kundi la wanamgambo la Jaish-e-Mohammad, lililoripotiwa kuhusika na shambulizi la bomu la Kashmir."
6107,"Lissu ameongeza kwamba “Kwa kutumia njia hizo za amri za kisiasa, vyombo vya ulinzi na usalama vikiwemo jeshi la wananchi la Tanzania, jeshi la polisi, taasisi ya kuzuia na kupambana na rushwa, na idara ya usalama wa taifa pamoja na mamlaka ya kodi TRA, vinatumika kukamata kwa nguvu na kutaifisha mali za wafanyabiashara wetu wa ndani na makampuni ya wawekezaji kutoka nje."
3191,"SAA, shirika kubwa la ndege la Afrika liliingia katika mpango wa kujilinda kutokana na hali ya kufilisika mwezi Disemba 2019, na tangu wakati huo lililazimika kusitisha safari zake zote za abiria kutokana na janga la virusi vya korona kote duniani."
4993,Shirika la habari la China Xinhua linasema kutakuwa na karibu safari 200 za ndege kuingia na kutoka Wuhan Jumatano.


Putting this together:

In [20]:
def search(X, query, num_results=10):
    score = cosine_similarity(X, query).flatten()
    idx = np.argsort(-score)[:num_results]
    return df.iloc[idx]

#### TF-IDF Vectorization

Minimizes the effect of insignificant words

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer()
X = tfv.fit_transform(df.documents)
q = tfv.transform([query])
names = tfv.get_feature_names_out()

In [22]:
df_docs = pd.DataFrame(X.toarray(), columns=names)
first_doc = df_docs.loc[0]
first_doc[first_doc != 0]

14            0.311283
19            0.260709
afya          0.234890
covid         0.273393
imeripoti     0.327010
jumatatu      0.253012
kuwa          0.143414
maambukizi    0.222536
takriban      0.275575
tanzania      0.210560
wamepata      0.413453
watu          0.161859
wizara        0.255201
ya            0.216513
zaidi         0.186419
Name: 0, dtype: float64

Instead of simple counts, we have a score showing the importance of each term. Perform a search:

In [23]:
search(X, q)

,documents
664,"Amesema ""mwanzoni tulizungumza kujifunza kutoka uzoefu wa nchi nyingine katika kupambana na janga la corona, sasa tunazungumzwa na nchi nyingine kama kielelezo hasi cha vita dhidi ya janga la corona Afrika na duniani."
4808,"Amesema ""mwanzoni tulizungumza kujifunza kutoka uzoefu wa nchi nyingine katika kupambana na janga la corona, sasa tunazungumzwa na nchi nyingine kama kielelezo hasi cha vita dhidi ya janga la corona Afrika na duniani."
2574,"Mali, nchi yenye huduma mbaya za afya, iliandaa uchaguzi jumapili, licha ya janga la virusi vya Corona."
6369,"Mkurugenzi Mkuu wa Shirika la Afya Duniani Tedros Adhanom Ghebreyesus, ameonya kufungwa kwa mipaka ya nchi na kusitisha shughuli zote ili kupambana na janga la COVID 19 kunaweza kusababisha kuongezeka kwa vifo kutokana na ugonjwa wa Malaria katika nchi za Afrika."
7116,China imesema Jumatano kuwa uamuzi wa Rais wa Marekani Donald Trump kusitisha ufadhili kwa Shirika la Afya Duniani kutaziathiri nchi zote wakati dunia ikikabiliwa na hatua muhimu ya kupambana na janga la virusi vya corona.
2311,"Hatua hiyo imechukuliwa huku maambukizi yakiendelea kuongezeka kote duniani, na baada ya Shirika la Afya Duniani (WHO) kutangaza maambukizi ya Corona kuwa janga la kimataifa."
3022,"“Inasikitisha kwamba nchi kadhaa duniani na mashirika mbalimbali, ikiwemo Marekani, China na shirika la afya duniani, yanakabiliwa na tishio la maambukizi ya Corona."
814,"Katika ukosoaji wa nadra kwa umma, shirika la afya Duniani-WHO wiki iliyopita lilieleza kwamba katika kupingana na kanuni za kimataifa za afya, Tanzania ilikataa kutoa taarifa za kina za kesi zinazoshukiwa kuwa za Ebola."
911,"Wakati huo huo, Museveni ametangaza mipango ya kuwarudisha Uganda raia wa nchi hiyo ambao wamekwama nchi za nje kutokana na janga la Corona."
1020,Kwa mujibu wa shirika la habari la Uingereza Reuters hisa za shirika la ndege la Ujerumani Luftansa zilipanda kwa asilimia 6.


### Embeddings

Moving beyond exact text matches to matching related concepts.

#### Singular Value Decomposition (SVD)

A mathematical technique that operates on a matrix to extract it's most important features. When used on a matrix representing the bag of words, it extracts association between words, thereby grouping words that are related close together in the vector space.

We use the vector representation from the TF-IDF vectorizer to create embeddings:

In [24]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500)
X_emb = svd.fit_transform(X)

In [25]:
X_emb.shape

(7672, 500)

Create query embeddings:

In [26]:
q

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 12 stored elements and shape (1, 19056)>

In [27]:
q_emb = svd.transform(q)
q_emb.shape

(1, 500)

Search the documents using SVD embeddings:

search(X_emb, q_emb)

#### BERT

The previous methods all used the bag-of-words approach, i.e. the order of words wasn't taken into account. BERT is a transformer model that accounts for order of words. We'll use a pretrained BERT model fine-tuned on swahili.

We'll need the tokenizer to convert the text into vectors according to the model's vocabulary, then the model will create embeddings.

In [33]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("flax-community/bert-swahili-news-classification")
model = AutoModelForSequenceClassification.from_pretrained("flax-community/bert-swahili-news-classification")

/home/krm/projects/search-engine/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Using two documents from our text corpus, we run through the process of creating embeddings:

In [87]:
texts = df.documents[:2].tolist()
encoded_text = tokenizer(texts, padding=True, return_tensors='pt')

In [88]:
encoded_text.input_ids.shape, encoded_text.input_ids

(torch.Size([2, 21]),
 tensor([[ 1057,   117,   902,   117,   367,   362,  2867,  3731,   200,    20,
            283,  5271,   869,   349,  8119,  4252,   117,  7632,    21,   588,
             22],
         [15117,  7672,   587,   156,  1938,   115,   367,    20,   870,  1374,
            544,    21,   595,    21,   662,   119,   628,   969,  1617,    22,
              0]]))

In [72]:
import torch

model.config.output_hidden_states=True
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_text)
    last_hidden_states = outputs.hidden_states[-1]

last_hidden_states.shape

torch.Size([2, 21, 768])

In [89]:
compressed_emb = last_hidden_states.mean(dim=1)
compressed_emb.shape, compressed_emb.numpy()

(torch.Size([2, 768]),
 array([[ 1.4280016 , -0.18435301, -0.03840554, ..., -1.2254812 ,
         -0.68361926,  0.7766203 ],
        [ 0.8678242 , -0.80565315, -0.9088004 , ..., -0.47340122,
          0.38405433,  0.7781875 ]], dtype=float32))

Since we have have a large dataset, we'll do inference in batches. We create a function for this.

In [106]:
from tqdm import tqdm

def get_embeddings(documents, batch_size=100):
    embedding_batches = []
    for i in tqdm(range(0, len(documents), batch_size)):
        batch = documents[i:i+batch_size]
        encoded_text = tokenizer(batch, padding=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**encoded_text)
            last_hidden_states = outputs.hidden_states[-1]
            embedding_batches.append(last_hidden_states.mean(dim=1).numpy())
    return np.vstack(embedding_batches)    

In [107]:
X_emb = get_embeddings(df.documents.to_list())

100%|███████████████████████████████████████████████████████████████████| 77/77 [29:11<00:00, 22.74s/it]


We also convert the query into embeddings:

In [111]:
q_emb = get_embeddings([query], batch_size=1)

100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 19.17it/s]


In [116]:
X_emb.shape, q_emb.shape

((7672, 768), (1, 768))

We then perform a search using BERT embeddings:

In [118]:
search(X_emb, q_emb)

,documents
5679,"Nchi tatu za Africa zinamaambukizi zaidi katika mlipuko wa virusi vya corona bara la Afrika, lakini naibu mkurugenzi wa Vituo vya Kudhibiti Magonjwa na Kinga Afrika wanasema bara lote lazima lichukue hatua."
3020,"“Umoja wa Afrika unaunga kwa dhati juhudi za WHO katika kupambana na virusi vya Corona na kutaka viongozi wote wa dunia, kuungana katika kuzuia maambukizi na vifo kutokana na virusi hivo” ameandika Mahamat kwenye mtandao huo."
6369,"Mkurugenzi Mkuu wa Shirika la Afya Duniani Tedros Adhanom Ghebreyesus, ameonya kufungwa kwa mipaka ya nchi na kusitisha shughuli zote ili kupambana na janga la COVID 19 kunaweza kusababisha kuongezeka kwa vifo kutokana na ugonjwa wa Malaria katika nchi za Afrika."
6932,"Mkuu wa Shirika la Afya, WHO, nchini Burundi alifukuzwa wiki iliyopita baada ya kueleza wasiwasi wake juu ya uelewa wa serikali unavyokinzana na hatari inayoletwa na virusi vya corona."
2536,"Shirika la Afya Duniani, WHO, limeripoti kuchukuwa hatua za haraka kukabiliana na mlipuko wa virusi vya Ebola nchini Uganda, huku hatua za madhubuti zikichukuliwa kuhakikisha kwamba watu wanaoishi kwenye mpaka wa Uganda na Jamhuri ya Kidemokrasia ya Congo wanaanza kupokea chanjo dhidi ya Ebola Hadi sasa, visa vitatu vya maambukizi vimethibitishwa ikiwemo kifo kimoja."
3022,"“Inasikitisha kwamba nchi kadhaa duniani na mashirika mbalimbali, ikiwemo Marekani, China na shirika la afya duniani, yanakabiliwa na tishio la maambukizi ya Corona."
7387,Ushauri wa WHO Jumanne wakurugenzi wakuu wa WHO walikutana na kutoa ushauri wa kuzitaka nchi zote duniani kushirikiana katika kupashana habari na kubadilishana takwimu juu ya virusi vya Corona.
5145,"Wakati timu za wanaokabiliana na Ebola wakihangaika kudhibiti mlipuko wa ugonjwa huo huko Jamhuri ya Kidemokrasia ya Congo, DRC, mawaziri wa afya wa Jumuiya ya Afrika Mashariki (EAC) wanasema wanatambua hatari zake na wanatathmini kuanza kutumia chanjo ya Ebola iliyoko katika majaribio."
3626,"Wataalam wa Shirika la Afya Duniani, WHO, wametoa mwelekeo mpya wa kutoa chanjo katika juhudi za kupambana na mlipuko wa Ebola katika Jamhuri ya Kidemokrasia ya Congo, DRC."
2460,"Shirika la Afya Duniani (WHO) Ijumaa limeonya kuwa watu 190,000 wanaweza kupoteza maisha mwaka 2020 barani Africa, iwapo serikali zitashindwa kudhibiti maambukizi ya virus vya corona."


In [119]:
query

'nchi za afrika zinajadiliana na shirika la afya kutafuta njia za kukabiliana na janga la corona'